## Tensorflow Serving Version 1.13 Example

Train and export a simple Softmax Regression TensorFlow model.
The model is from the TensorFlow "MNIST For ML Beginner" tutorial. This program
simply follows all its training instructions, and uses TensorFlow SavedModel to
export the trained model with proper signatures that can be loaded by standard
tensorflow_model_server.
<br>
Usage: mnist_saved_model.py [--training_iteration=x] [--model_version=y] export_dir

In [1]:
from __future__ import print_function

import os
import sys
import pickle
import numpy as np
import tensorflow as tf

In [2]:
def load(data_path):
    with open(data_path,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

class MnistData:
    def __init__(self, filenames, need_shuffle, datatype='training'):
        all_data = []
        all_labels = []
        x_train, y_train, x_test, y_test = load(filenames) #"data/mnist.pkl"
        
        if datatype=='training':
            self._data = x_train / 127.5 -1
            self._labels = y_train
            print(self._data.shape)
            print(self._labels.shape)
        else:
            self._data = x_test / 127.5 -1
            self._labels = y_test
            print(self._data.shape)
            print(self._labels.shape)
            
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

In [3]:
train_filenames = "../4_basic_image_recognition/data/mnist.pkl"
train_data = MnistData(train_filenames, True, 'training')
test_data = MnistData(train_filenames, False, 'test')

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [4]:
tf.app.flags.DEFINE_integer('training_iteration', 1000, 'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('work_dir', './', 'Working directory.')
FLAGS = tf.app.flags.FLAGS
export_path_base = ('./model')
batch_size = 50

In [5]:
def main(_):
    xs = tf.placeholder(tf.float32, [None, 28*28], name='x')
    ys = tf.placeholder(tf.int64, [None], name = 'y')

    W = tf.get_variable('w', [xs.get_shape()[-1], 10], initializer=tf.random_normal_initializer(0, 1))
    b = tf.get_variable('b', [10], initializer=tf.constant_initializer(0.0))
    y_ = tf.matmul(xs, W) + b
    p_y = tf.nn.softmax(y_)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=ys, logits=y_)
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    init = tf.global_variables_initializer()
    predict = tf.argmax(p_y, 1)
    correct_prediction = tf.equal(predict, ys)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    
    with tf.Session() as sess:
        sess.run(init)
        for i in range(FLAGS.training_iteration):
            batch_data, batch_labels = train_data.next_batch(batch_size)
            loss_val, acc_val, _ = sess.run([loss, accuracy, train_step], feed_dict={xs: batch_data, ys: batch_labels})
            if (i+1) % 1000 == 0:
                print('[Traini] Step: %d, loss: %4.5f, acc: %4.5f' %(i+1, loss_val, acc_val))

            if (i+1) %10000 == 0:
                all_test_acc_val = []
                for j in range(test_steps):
                    test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                    test_acc_val = acc = sess.run(accuracy, feed_dict={xs: test_batch_data, ys: test_batch_labels})
                    all_test_acc_val.append(test_acc_val)
                test_acc = np.mean(all_test_acc_val)
                print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))
        print('Done training!')
    
    
        export_path = os.path.join(
            tf.compat.as_bytes(export_path_base),
            tf.compat.as_bytes(str(FLAGS.model_version)))

        print('Exporting trained model to', export_path)
        
        builder = tf.saved_model.builder.SavedModelBuilder(export_path)

        serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
        values, indices = tf.nn.top_k(p_y, 10)
        table = tf.contrib.lookup.index_to_string_table_from_tensor(tf.constant([str(i) for i in range(10)]))
        prediction_classes = table.lookup(tf.to_int64(indices))
        
        print('serialized_tf_example: ', serialized_tf_example)
        print('prediction_classes: ', prediction_classes)
        print('values: ', values)
        print('indices: ', indices)

        # Build the signature_def_map.
        classification_inputs = tf.saved_model.utils.build_tensor_info(serialized_tf_example)
        classification_outputs_classes = tf.saved_model.utils.build_tensor_info(prediction_classes)
        classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)
        
        print('classification_inputs', classification_inputs)
        print('classification_outputs_classes', classification_outputs_classes)
        print('classification_outputs_scores', classification_outputs_scores)

        classification_signature = (
            tf.saved_model.signature_def_utils.build_signature_def(
                inputs={
                  tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                      classification_inputs
                },
                outputs={
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                      classification_outputs_classes,
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                      classification_outputs_scores
                },
                method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME
            )
        )

        tensor_info_x = tf.saved_model.utils.build_tensor_info(xs)
        tensor_info_y = tf.saved_model.utils.build_tensor_info(ys)

        prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'images': tensor_info_x},
            outputs={'scores': tensor_info_y},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
        )
        
        print('tensor_info_x', tensor_info_x)
        print('tensor_info_y', tensor_info_y)
        print('classification_signature', classification_signature)
        print('prediction_signature', prediction_signature)

        builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                'predict_images':
                  prediction_signature,
                tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                  classification_signature,
            },
            main_op=tf.tables_initializer(),
            strip_default_attrs=True)

        builder.save()

        print('Done exporting!')

In [6]:
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[Traini] Step: 1000, loss: 1.05026, acc: 0.88000
Done training!
Exporting trained model to b'./model/1'

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
serialized_tf_example:  Tensor("tf_example:0", dtype=string)
prediction_classes:  Tensor("index_to_string_Lookup:0", shape=(?, 10), dtype=string)
values:  Tensor("TopKV2:0", shape=(?, 10), dtype=float32)
indices:  Tensor("TopKV2:1", shape=(?, 10), dtype=int32)
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
classification_inputs name: "tf

SystemExit: 

/home/bai/.virtualenvs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Start Tensorflow Serving (don't specify model version)

docker run -p 8501:8501 --mount type=bind,source=/home/bai/PycharmProjects/tensorflow_for_enginner/11_tf_serving/model/1,target=/models/my_model -e MODEL_NAME=my_model -t tensorflow/serving